In [1]:
# import packages
!pip install faker
!pip install names
!pip uninstall -y dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl
!pip install dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl

import json
import pandas as pd
import faker
import os
from IPython.display import display
import requests
import random
import ipywidgets as wid
import numpy as np

Uninstalling OpenSpecimenAPIconnector-0.9.1:
  Successfully uninstalled OpenSpecimenAPIconnector-0.9.1
Processing ./dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl


In [2]:
import OpenSpecimenAPIconnector.os_core as os_core
import OpenSpecimenAPIconnector.os_util as os_util
import OpenSpecimenAPIconnector.mg_core as mg_core
import OpenSpecimenAPIconnector.mg_util as mg_util

In [3]:
def csv_download_link(df, csv_file_name, delete_prompt=False):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    from IPython.core.display import display, HTML
    link = "http://localhost:8888/edit/work/" + csv_file_name
    df.to_csv(csv_file_name, index=False)
    text = "Download/Edit " + csv_file_name
    display(HTML("<a href={} target=_blank>{}</a>".format(link, text)))
   
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [4]:
def multi_checkbox_widget(options_dict):
    """ Widget with a search field and lots of checkboxes """
    search_widget = wid.Text()
    output_widget = wid.Output()
    options = [x for x in options_dict.values()]
    options_layout = wid.Layout(
        overflow='auto',
        border='1px solid black',
        width='300px',
        height='300px',
        flex_flow='column',
        display='flex'
    )
    
    #selected_widget = wid.Box(children=[options[0]])
    options_widget = wid.VBox(options, layout=options_layout)
    #left_widget = wid.VBox(search_widget, selected_widget)
    multi_select = wid.VBox([search_widget, options_widget])

    @output_widget.capture()
    def on_checkbox_change(change):
        
        selected_recipe = change["owner"].description
        #print(options_widget.children)
        #selected_item = wid.Button(description = change["new"])
        #selected_widget.children = [] #selected_widget.children + [selected_item]
        options_widget.children = sorted([x for x in options_widget.children], key = lambda x: x.value, reverse = True)
        
    for checkbox in options:
        checkbox.observe(on_checkbox_change, names="value")

    # Wire the search field to the checkboxes
    @output_widget.capture()
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            # Reset search field
            new_options = sorted(options, key = lambda x: x.value, reverse = True)
        else:
            # Filter by search field using difflib.
            #close_matches = difflib.get_close_matches(search_input, list(options_dict.keys()), cutoff=0.0)
            close_matches = [x for x in list(options_dict.keys()) if str.lower(search_input.strip('')) in str.lower(x)]
            new_options = sorted(
                [x for x in options if x.description in close_matches], 
                key = lambda x: x.value, reverse = True
            ) #[options_dict[x] for x in close_matches]
        options_widget.children = new_options

    search_widget.observe(on_text_change, names='value')
    display(output_widget)
    return multi_select

In [5]:
def display_search_select_box(cps_dict):
    descriptions = [key for key in  cps_dict.keys()]

    options_dict = {
        x: wid.Checkbox(
            description=x, 
            value=False,
            style={"description_width":"0px"}
        ) for x in descriptions
    }

    def f(**args):

        results = [key for key, value in args.items() if value]
        display(results)

    ui = multi_checkbox_widget(options_dict)
    out = wid.interactive_output(f, options_dict)
    display(wid.HBox([ui, out]))
    return ui

In [6]:
# The URL of the Openspecimen
print('Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter')
#base_url='http://localhost:9000' # input() set to hardcoded at the moment just change below or add urls for more instances
base_url='http://biobank.silicolab.bibbox.org'
base_url2= 'http://biobank-7-2.silicolab.bibbox.org' 
base_url+='/openspecimen/rest/ng'
base_url2+='/openspecimen/rest/ng'
print('The whole URL for API Calls is: ' + base_url)

Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter
The whole URL for API Calls is: http://biobank.silicolab.bibbox.org/openspecimen/rest/ng


In [7]:
# Loginname and Pasword of the API-User, recommended is an extra API User
print('Loginname of the user:')
loginname='admin'#input() 
print('Password of the user:')
password='Login@123'#input()

Loginname of the user:
Password of the user:


The next step is to choose the entity which will be created or updated. The following schemas are in Openspecimen available: (Name, openspecimen entity). One have to write the openspecimen entity. ('Collection Protocol', 'cp') ('Collection Protocol Registration', 'cpr') ('User', 'user') ('User Roles', 'userRoles') ('Site', 'site') ('Shipments', 'shipment') ('Institute', 'institute') ('Distribution Protocol Requirement', 'dpRequirement') ('Distribution Protocol', 'distributionProtocol') ('Distribution Order', 'distributionOrder') ('Containers', 'storageContainer') ('Container Type', 'storagecontainerType') ('Container Shipment', 'containerShipment') ('Collection Protocol Event', 'cpe') ('Master Specimen', 'masterSpecimen') ('Participant', 'participant') ('Specimen Requirement', 'sr') ('Visit', 'visit') ('Specimen Aliquot', 'specimenAliquot') ('Specimen Derivate', 'specimenDerivative') ('Specimen Disposal', 'specimenDisposal') ('Consent', 'consent')

In [8]:
schema=wid.Dropdown(
    options= [(), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'), ('Collection Protocol Registration', 'cpr'),
              ('User', 'user'), ('User Roles', 'userRoles'), ('Site', 'site'), ('Shipments', 'shipment'), 
              ('Institute', 'institute'), ('Distribution Protocol Requirement', 'dpRequirement'), 
              ('Distribution Protocol', 'distributionProtocol'), ('Distribution Order', 'distributionOrder'), 
              ('Containers', 'storageContainer'), ('Container Type', 'storagecontainerType'),
              ('Container Shipment', 'containerShipment'), ('Collection Protocol Event', 'cpe'),
              ('Master Specimen', 'masterSpecimen'), ('Participant', 'participant'), ('Specimen Requirement', 'sr'),
              ('Visit', 'visit'), ('Specimen Aliquot', 'specimenAliquot'), 
              ('Specimen Derivate', 'specimenDerivative'),('Specimen Disposal', 'specimenDisposal'),
              ('Consent', 'consent')],
    value=(),
    description='Entity:')

operation = wid.Dropdown(
    options = ["CREATE",  'UPDATE'],
              value="CREATE",
              description='Operation:')

sel_schema = schema.value
def change_value_schema(change):
    global sel_schema
    if change.new:
        sel_schema = change.new[1]
    else: sel_schema = None

op = operation.value
def change_value_op(change):
    global op
    if change.new:
        op = change.new
    else: op = None

schema.observe(change_value_schema, names="value")
operation.observe(change_value_op, names="value")

print('Choose your Entity:')
display(schema)
print('Choose Operation:')
display(operation)

Choose your Entity:


Dropdown(description='Entity:', options=((), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'), ('Collec…

Choose Operation:


Dropdown(description='Operation:', options=('CREATE', 'UPDATE'), value='CREATE')

In [9]:
print("Selected Schema: ", sel_schema)
print("Selected Operation: ", op)

#initialize csv_bulk, bulk_operations and CSV_exporter
csv_files= os_core.csv_bulk(base_url2,(loginname, password))
bulk_op= os_util.bulk_operations(base_url2, (loginname, password))
csv_export = os_util.Export_OP(base_url2, (loginname, password))

Selected Schema:  visit
Selected Operation:  CREATE


In [10]:
# get necessary cp information
cp_tools = os_core.collection_protocol(base_url2, (loginname, password))

# get cp info and wirte a dict in order to simplify the api output
cps = cp_tools.get_all_collection_protocols()
cps_dict = {}
for i, item in enumerate(cps):
    # print("Cp Title:")
    # print(item["title"])
    # print("Cp ID: ",item["id"], "\n")
    cps_dict[item["title"]] = item["id"]

In [11]:
# choose target CP / Check if matching is correct; ui is needed for output

ui = display_search_select_box(cps_dict)

Output()

In [12]:
# get selected cp_ids
cp_ids = []
selected_options = [widget.description for widget in ui.children[1].children if widget.value]
for item in selected_options:
    cp_ids.append(cps_dict[item])

print("CP ids selected for further usage: ", cp_ids)
print()

# Download Template file to see available fields 
# CP Registration CSV are downloaded for the aggregtation example 
# in order to get participant age data for aggregating age fields in the given example:
export_cps_csv = []
export_cps_def = []
for i, cp_id in enumerate(cp_ids):
    # Example for downloading tempalte csv for differen entities 
    if i == 0:
        template = csv_files.get_template(schemaname=sel_schema)
        template_df =  pd.DataFrame(template)
        print("Template CSV of {}:".format(sel_schema))
        display(template_df)
    
    export_cps_csv.append(csv_export.export_file(sel_schema, cp_id))
    # display the downladed csv for the given entity
    if i == 0:
        print("Example CSV of {} with data:".format(sel_schema))
        display(export_cps_csv[-1])
    export_cps_def.append(cp_tools.get_cp_def(cp_id))

CP ids selected for further usage:  [17, 18]

Template CSV of visit:


,Identifier,CP Short Title,PPID,Event Label,Name,Visit Date,Visit Site,Visit Status,Clinical Diagnosis#1,Clinical Diagnosis#2,Clinical Status,Cohort,Path. Number,Path Report Name,Missed/Not Collected Reason,Comments,Activity Status,Reason,Missed/Not Collected By#Email Address


Waiting for job to finish
Example CSV of visit with data:


,Identifier,CP Short Title,Visit Name,Specimen Requirement Code,Specimen Label,Barcode,Class,Type,Lineage,Parent Specimen Label,...,Collection Event#User#Email Address,Collection Event#Comments,Collection Event#Procedure,Collection Event#Container,Received Event#Date and Time,Received Event#User#Email Address,Received Event#Comments,Received Event#Quality,Specimen Additional Fields#Media Volume,Specimen Additional Fields#Cell Viability (%)
0,1,HBOOCA,visit1568,NaN,specimen1568,NaN,Fluid,Plasma,New,NaN,...,admin@localhost,NaN,Not Specified,Not Specified,12-01-2021 08:10,admin@localhost,NaN,Acceptable,NaN,NaN
1,2,HBOOCA,visit3288,NaN,specimen3288,NaN,Molecular,DNA,New,NaN,...,admin@localhost,NaN,Not Specified,Not Specified,12-01-2021 08:10,admin@localhost,NaN,Acceptable,NaN,NaN
2,3,HBOOCA,visit1943,NaN,specimen1943,NaN,Fluid,Plasma,New,NaN,...,admin@localhost,NaN,Not Specified,Not Specified,12-01-2021 08:10,admin@localhost,NaN,Acceptable,NaN,NaN
3,4,HBOOCA,visit1558,NaN,specimen1558,NaN,Fluid,Serum,New,NaN,...,admin@localhost,NaN,Not Specified,Not Specified,12-01-2021 08:10,admin@localhost,NaN,Acceptable,NaN,NaN
4,5,HBOOCA,visit1549,NaN,specimen1549,NaN,Fluid,Serum,New,NaN,...,admin@localhost,NaN,Not Specified,Not Specified,12-01-2021 08:10,admin@localhost,NaN,Acceptable,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3309,3310,HBOOCA,visit2357,NaN,specimen2357,NaN,Fluid,Serum,New,NaN,...,admin@localhost,NaN,Not Specified,Not Specified,12-01-2021 08:24,admin@localhost,NaN,Acceptable,NaN,NaN
3310,3311,HBOOCA,visit1783,NaN,specimen1783,NaN,Fluid,Serum,New,NaN,...,admin@localhost,NaN,Not Specified,Not Specified,12-01-2021 08:24,admin@localhost,NaN,Acceptable,NaN,NaN
3311,3312,HBOOCA,visit847,NaN,specimen847,NaN,Molecular,DNA,New,NaN,...,admin@localhost,NaN,Not Specified,Not Specified,12-01-2021 08:24,admin@localhost,NaN,Acceptable,NaN,NaN
3312,3313,HBOOCA,visit1132,NaN,specimen1132,NaN,Molecular,DNA,New,NaN,...,admin@localhost,NaN,Not Specified,Not Specified,12-01-2021 08:24,admin@localhost,NaN,Acceptable,NaN,NaN


In [15]:
# in case of multiple targets; just an example for Collection Protocolls exists do far since additional 
# fields are attached there 
if sel_schema == "cpr":
    for h, cp in enumerate(export_cps_csv):
        age_array = []
        for date in cp["Date Of Birth"]:
            age_array.append(2020 - float(date.split("-")[-1]))
        age_array = np.array(age_array)
        print("Age high: ",np.max(age_array))
        print("Age low: ", np.min(age_array))
        print("Age mean: ", age_array.mean())
        cp_data = export_cps_def[h]
        for i, item in enumerate(cp_data[1]["extensionDetail"]["attrs"]):
            if cp_data["extensionDetail"]["attrs"][i]["name"] == "AgeLow":
                cp_data["extensionDetail"]["attrs"][i]["value"] = str(int(np.min(age_array)))
                cp_data["extensionDetail"]["attrs"][i]["displayValue"] = str(np.min(age_array))
            elif cp_data["extensionDetail"]["attrs"][i]["name"] == "AgeHigh":
                cp_data["extensionDetail"]["attrs"][i]["value"] = str(int(np.max(age_array)))
                cp_data["extensionDetail"]["attrs"][i]["displayValue"] = str(np.max(age_array))
        # print("Current cp update:") 
        # print(cp["extensionDetail"]["attrs"])
        # input()
else:
    print("In this Demo the only fields to aggregate are attached to Collection Protocols")

PK  �)AR            
��2E�A�d�u�۫)
�A��8��toڧ�&��u8��q�r%���r0��k��7\s�u���X��r��
VR9膻kRf��Y��ה*�1ey������k�+qYn��F�g��-��m+
�%�ŕo�"�a�$ӵ��{+�D��v��e�[ѧ+N����bl��{Ǖ�(36A�5Εخ�0�"�a���51�m���ɲ���يbl���]n=��[I������[Q��%��2q��)2]a�tN�[�u��KVo���|�.Ē%[�Dd��8��u5ܹc£�'��M�����O���� �����zz�5p�	:�Q��i��M-?l��p%u��H���PA�+�qt�[�+N�d���"tdo����LQ��r%^�B���&��i�fLQ�%U��
�c�>U2:���ڊ�k� ��Z�:`�l�+�[��5AT�TɤB�,6A� Ε�v��ڊ���z���p��>Z�PI1ἰl��8YVN8�w��0X��*�A��A+O��9������-V��10�m���A��7\�U| 0E\�\��*�.��0X����aZ[ї+N�$�^fn~yd��ɂ%��N,(T����}W�
06E`�`IQ��&ꙢH'K�
v���J�f`?gQCg+�q�c�A���\�y�|�M�O�x��F=������֗X��;��z�U[�ˀ)�l�����k�g��8Yb�
Fح���J�	/\����0Xb��[g�BA�,Is]��0<�:�'�J�\G�lE\�\ɨB\��p����Jr\�׭l,)*�8p+Kl�:�Jk+
�Q��ú�������zӵ�p�0K�[���nD5J����6p�"�Q�dl�l7���;�$���o�U��@���Z�A����ڼ�j���6�`��:qɩ1$Y��g�IOU���`I�V[�,0E`�`��:X�5��;�k��ӡb�4W�΂ǵ��&��R�>���g�,X��:��e+
�7�?�R%f�<]���OV�+Q�*�i�"�Q�lD�W,S�